# Capítulo 2 - Datasets de Referencia y Comparación de Modelos

Tras la introducción conceptual al problema de la detección de anomalías, este capítulo se centra en los **datasets que servirán de base para la validación y comparación de modelos**.  

Dado que los datos procedentes de archivos `.MDF` no cuentan con etiquetas fiables ni anotaciones de anomalías, resulta necesario apoyarse en **conjuntos de datos de referencia** que sí disponen de señales temporales y etiquetas de anomalías previamente definidas. Estos datasets permitirán evaluar los algoritmos en un entorno controlado antes de trasladar los métodos a señales más complejas y sin etiquetas.

## Datasets utilizados

Los datasets seleccionados provienen de distintos dominios (procesos industriales, redes de sensores y series sintéticas), lo cual aporta diversidad y robustez a la validación:

- **BATADAL**: señales de una planta de agua potable, con periodos normales y bajo ataque.  
- **SKAB**: datos de control industrial con válvulas, incluye escenarios de funcionamiento normal y fallos.  
- **WADI**: simulaciones de un sistema de distribución de agua con múltiples ataques cibernéticos.  
- **EbayRanSynCoders**: dataset sintético con patrones aleatorios y anomalías inyectadas.  
- **SMAP**: telemetría de satélites, con etiquetas de anomalías reales.  
- **MSL**: telemetría del satélite *Mars Science Laboratory*, también con anotaciones de anomalías.  

Cada uno de estos conjuntos se dividirá en **entrenamiento, validación y prueba**, garantizando consistencia en los experimentos.

## Consideraciones sobre licencias y distribución de datasets

Los datasets empleados en este trabajo provienen de fuentes abiertas y repositorios públicos.  
A continuación se detallan sus orígenes y las licencias asociadas:

| Dataset             | Fuente                                                                                      | Licencia                                                                 | Uso permitido en el trabajo |
|---------------------|---------------------------------------------------------------------------------------------|--------------------------------------------------------------------------|-----------------------------|
| **SKAB**            | [Kaggle - Skoltech Anomaly Benchmark](https://www.kaggle.com/datasets/yuriykatser/skoltech-anomaly-benchmark-skab) | GNU Affero General Public License 3.0 (AGPL-3.0)                          | ✅ Se puede usar y citar, **no redistribuir los datos en repositorios cerrados**. |
| **CATS**            | [Kaggle - Controlled Anomalies Time Series](https://www.kaggle.com/datasets/patrickfleith/controlled-anomalies-time-series-dataset) | Creative Commons Attribution 4.0 (CC BY 4.0)                              | ✅ Libre uso y redistribución, siempre que se cite la fuente. |
| **WaDI**            | iTrust, Singapore University of Technology and Design                                       | Propiedad iTrust (descarga académica con registro)                        | ✅ Uso permitido para investigación académica. No redistribuir. |
| **BATADAL**         | iTrust, Singapore University of Technology and Design                                       | Propiedad iTrust (descarga académica con registro)                        | ✅ Uso permitido para investigación académica. No redistribuir. |
| **Ebay RANSynCoders** | [GitHub - eBay RANSynCoders](https://github.com/eBay/RANSynCoders)                         | BSD License (Copyright © 2021 eBay Inc.)                                  | ✅ Uso, modificación y redistribución permitida con atribución y disclaimers. |
| **SMAP & MSL**      | [Kaggle - NASA Anomaly Detection Datasets](https://www.kaggle.com/datasets/patrickfleith/nasa-anomaly-detection-dataset-smap-msl) | Datos originales © autores de NASA. Redistribuidos en Kaggle.             | ✅ Uso académico permitido con cita. Redistribución limitada. |

### Implicaciones prácticas

- En este trabajo **se describen y utilizan** los datasets, pero **no se redistribuyen directamente los ficheros originales**.  
- Para su acceso, se proporcionan los enlaces oficiales de descarga.  
- El procesamiento intermedio realizado (conversión a `.parquet` mediante la clase `DatasetsToParquet`) **no altera el contenido original**, solo su formato, y por tanto no modifica los términos de las licencias.  
- Los gráficos, tablas y resultados generados a partir de los datasets **sí pueden incluirse libremente en este trabajo**, siempre citando la fuente correspondiente.  

## Unificación y preprocesado

Con el fin de trabajar de manera homogénea con todos los datasets, se desarrolló una clase en Python llamada **`DatasetsToParquet`**, cuyo objetivo es:

1. **Cargar** los datos originales desde su formato nativo (CSV, NPY, etc.).  
2. **Estandarizar** columnas clave como `timestamp` y `anomaly`.  
3. **Dividir** los datos en *training*, *validation* y *test* (con un split 60%-40% en validación y prueba).  
4. **Guardar** los resultados en formato **`.parquet`**, lo que facilita el acceso eficiente y la compatibilidad con librerías como `pandas` y `pyarrow`.

De esta manera, todos los conjuntos quedan listos para ser utilizados en la evaluación de algoritmos, reduciendo la complejidad derivada de la heterogeneidad de los formatos originales.

## Consideraciones sobre la validación en datos temporales

Estos datasets contienen **series temporales**, por lo que no es adecuado aplicar validaciones que mezclen pasado y futuro (p. ej., **k-fold** aleatorio o **bootstrap** estándar), ya que introducirían **fuga de información**.

En su lugar, la validación se plantea con **splits temporales que respetan el orden cronológico**. A lo largo del trabajo se usarán dos esquemas sencillos y prácticos (descritos en detalle en el **Capítulo 4**):

- **Time Series Cross-Validation (TSCV) con ventana creciente (Expanding)**: se entrena con un bloque inicial y, en cada split, se amplía el historial antes de validar en el siguiente tramo.  
- **Walk-Forward con reentrenamiento**: se entrena con el bloque disponible, se valida en el siguiente y, tras evaluar, se incorpora ese bloque al entrenamiento antes de avanzar.

En resumen, el flujo es:
1. **Entrenamiento** (solo con datos pasados).  
2. **Validación** (evaluación en el siguiente bloque temporal, sin mezclar futuro con pasado).  
3. **Test** (medición final en datos no usados previamente).

Los detalles operativos (tamaños de bloques, deslizamiento, limpieza previa de datos, etc.) se concretan en el **Capítulo 4**.

## Estrategias de comparación de modelos

Para comparar distintos enfoques de detección de anomalías, se empleará un conjunto de **métricas complementarias**. La definición exacta y el modo de cálculo (a nivel de punto o de intervalo) se especifica en el **Capítulo 4**. Entre ellas:

- **NAB Score (Numenta Anomaly Benchmark):** prioriza detecciones tempranas y penaliza falsas alarmas.  
- **Precision y Recall:** miden la calidad de la detección bajo un umbral definido.  
- **ROC-AUC:** área bajo la curva *Receiver Operating Characteristic*, que evalúa la capacidad de separación entre clases.  
- **Cobertura de ventanas:** mide qué proporción de las ventanas anómalas definidas quedan cubiertas por las detecciones del modelo.  


Además, se considerará la **consistencia entre datasets** (robustez) como indicador de generalización.

## Próximos pasos

En el siguiente capítulo se presentarán los **métodos de detección** que se evaluarán sobre estos datasets. La **metodología de validación** y el **cálculo de métricas** se detallan íntegramente en el **Capítulo 4**.
